### Generate a Complete Database 

- date : 24/05/2024
- New features : generate a complete database using "OrdalieTech/Solon-embeddings-large-0.1" embedding model (config file have been improved)
- Expected improvement : improving the retrieval capabilities with a much strong embedding model.

In [1]:
import sys
import os
import pandas as pd

### Building New Complete Dataset based on config files 

In [2]:
from db_building import  build_database_from_csv
#db = build_database_from_csv('/home/onyxia/work/llm-open-data-insee/data_complete.csv')
#db.similarity_search("Quels sont les chiffres du chômages en 2023")

### Loading Dataset based on config files

In [3]:
from db_building import reload_database_from_local_dir
db = reload_database_from_local_dir(persist_directory="/home/onyxia/work/llm-open-data-insee/data/chroma_db")

2024-06-17 13:13:55,348 - INFO - Load pretrained SentenceTransformer: OrdalieTech/Solon-embeddings-large-0.1
/opt/mamba/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-06-17 13:14:00,066 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-06-17 13:14:00,179 - INFO - Collection insee_data is not created.
2024-06-17 13:14:00,180 - INFO - The database (collection insee_data) has been reloaded from directory /home/onyxia/work/llm-open-data-insee/data/chroma_db


In [4]:
#check if there are at least one encoded document in our vectorstore
print(len(db.get()["ids"])) 

In [4]:
result = db.similarity_search("Quels résultats au BAC les étudiants de classes préparatoires ont ils généralement?", k = 5 )
print(result[0])

2024-06-17 13:14:23,618 - INFO - Start multi-process pool on devices: cuda:0


page_content='.  Une certification intermédiaire est intégrée au parcours en première professionnelle (obligatoire pour les scolaires, facultatif pour les apprentis). Près de neuf élèves sur dix passent ainsi un examen avant leur baccalauréat professionnel: un BEP pour 79% d’entre eux ou un CAP pour 8%. Les taux de réussite sont de plus de 80% et la quasi-totalité des lauréats poursuivent leur scolarité. Par contre, ceux qui échouent sont un peu moins nombreux à poursuivre: huit sur dix continuent, dont deux tiers en terminale professionnelle comme prévu, 10% redoublent la première et 5% se réorientent en CAP.  Après trois ans de scolarité, 67% des élèves entrés en seconde professionnelle trois ans plus tôt se présentent à l’examen du baccalauréat professionnel et plus de trois candidats sur quatre le réussissent. Le taux de réussite est plus faible dans les domaines de la production (74%), passant de 67% pour le domaine «génie civil, construction, bois» à 86% pour celui des «matériaux

In [29]:
from config import RAG_PROMPT_TEMPLATE, EMB_MODEL_NAME, MODEL_NAME
from model_building import build_llm_model
from chain_building.build_chain import (
    load_retriever,
    build_chain
    )

from langchain_core.prompts import PromptTemplate
import chainlit as cl

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def build_chain_test(retriever, prompt, llm):
    """
    Build a LLM chain based on Langchain package and INSEE data
    """
    # Create a Langchain LLM Chain
    chain = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | prompt
        | llm
        | StrOutputParser()
    )

    rag_chain_with_source = RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    ).assign(answer=chain)

    return rag_chain_with_source

In [30]:
prompt = PromptTemplate(input_variables=["context", "question"], template=RAG_PROMPT_TEMPLATE)
print(prompt)

input_variables=['context', 'question'] template="\n<s>[INST]\nTu es un assistant spécialisé dans la statistique publique répondant aux questions d'agent de l'INSEE.\nRéponds en Français seulement.\nUtilise les informations obtenues dans le contexte, réponds de manière argumentée à la question posée.\nLa réponse doit être développée et citer ses sources.\n\nSi tu ne peux pas induire ta réponse du contexte, ne réponds pas.\nVoici le contexte sur lequel tu dois baser ta réponse :\nContexte: {context}\n        ---\nVoici la question à laquelle tu dois répondre :\nQuestion: {question}\n[/INST]\n"


In [8]:
retriever = db.as_retriever(search_type="mmr", search_kwargs={"score_threshold": 0.5, "k": 5})

#retriever = load_retriever(emb_model_name=EMB_MODEL_NAME,
                        #persist_directory="/home/onyxia/work/llm-open-data-insee/data/chroma_db")

In [ ]:
llm = build_llm_model(model_name=MODEL_NAME,
                        quantization_config=True,
                        config=True, 
                        token = os.environ["HF_TOKEN"])

In [57]:
chain = build_chain_test(retriever, prompt, llm)

In [58]:
chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7efc7cace780>, search_type='mmr', search_kwargs={'score_threshold': 0.5, 'k': 5}),
  question: RunnablePassthrough()
}
| RunnableAssign(mapper={
    answer: RunnableAssign(mapper={
              context: RunnableLambda(lambda x: format_docs(x['context']))
            })
            | PromptTemplate(input_variables=['context', 'question'], template="\n<s>[INST]\nTu es un assistant spécialisé dans la statistique publique répondant aux questions d'agent de l'INSEE.\nRéponds en Français seulement.\nUtilise les informations obtenues dans le contexte, réponds de manière argumentée à la question posée.\nLa réponse doit être développée et citer ses sources.\n\nSi tu ne peux pas induire ta réponse du contexte, ne réponds pas.\nVoici le contexte sur lequel tu dois baser ta réponse :\nContexte: {context}\n        ---\nVoici la question à laquelle tu d

In [53]:
question = "Quel est le but initial derrière la création du système de retraites français après la Seconde Guerre mondiale?" 
#question = "Quelle est la cause principale de l'augmentation de l'indice des prix à la consommation (IPC)?"
results = retriever.invoke(question)

for i, doc in enumerate(results):
    print(f"Doc {i} : {doc.metadata["source"]}")
    print(doc.page_content)

2024-06-17 14:20:31,369 - INFO - Start multi-process pool on devices: cuda:0


Doc 0 : https://www.insee.fr/fr/information/5008705
Le système de retraites français est le fruit d’une volonté postérieure à la deuxième guerre mondiale de construire un système d’assurance vieillesse capable de couvrir l’ensemble de la population. Il s’inscrit dans l’objectif d’offrir aux Français un large système de couverture sociale, permettant de se prémunir de tous les risques: santé, retraites, famille et accidents du travail. À ses débuts, l’existence de catégories auxquelles étaient déjà versées des pensions de retraite, doublée des réserves émises par les travailleurs indépendants, a orienté les choix vers un système de retraites par catégories socioprofessionnelles. Il en a résulté une pluralité de régimes, pour la plupart toujours existants, de dimensions bien différentes – allant de quelques centaines de bénéficiaires pour le régime du port autonome de Strasbourg ou celui de la Comédie-Française, à plusieurs millions pour le régime général qui regroupe les salariés du sec

In [46]:
for chunk in chain.stream(question):
    print(chunk)

2024-06-17 14:10:06,543 - INFO - Start multi-process pool on devices: cuda:0


{'question': "Quelle est la cause principale de l'augmentation de l'indice des prix à la consommation (IPC)?"}


{'context': [Document(page_content='L’indice des prix à la consommation est l’instrument de mesure de l’inflation. Il permet d’estimer, entre deux périodes données, la variation du niveau général des prix des biens et des services proposés aux consommateurs sur le territoire. C’est une mesure synthétique des évolutions de prix à qualité constante. La quasi-totalité des biens et des services marchands proposés aux consommateurs fait partie du champ de l’indice. Il permet de revaloriser les pensions alimentaires, les loyers d’habitation, les rentes viagères, et d’indexer le Smic et les retraites, ainsi que le taux du livretA ou d’autres produits financiers. Il permet de suivre mois par mois l’évolution des prix et donc d’apprécier les tendances inflationnistes.  Publications ?', metadata={'authors': '', 'categories': 'Publications grand public', 'collections': 'Insee Flash', 'date_diffusion': '2024-03-26T15:00:00Z', 'insee_id': 8058872, 'intertitres': 'Évolution en glissement annuel de l

In [47]:
answer = chain.invoke(question) 

2024-06-17 14:11:39,216 - INFO - Start multi-process pool on devices: cuda:0


{'context': [Document(page_content='L’indice des prix à la consommation est l’instrument de mesure de l’inflation. Il permet d’estimer, entre deux périodes données, la variation du niveau général des prix des biens et des services proposés aux consommateurs sur le territoire. C’est une mesure synthétique des évolutions de prix à qualité constante. La quasi-totalité des biens et des services marchands proposés aux consommateurs fait partie du champ de l’indice. Il permet de revaloriser les pensions alimentaires, les loyers d’habitation, les rentes viagères, et d’indexer le Smic et les retraites, ainsi que le taux du livretA ou d’autres produits financiers. Il permet de suivre mois par mois l’évolution des prix et donc d’apprécier les tendances inflationnistes.  Publications ?', metadata={'authors': '', 'categories': 'Publications grand public', 'collections': 'Insee Flash', 'date_diffusion': '2024-03-26T15:00:00Z', 'insee_id': 8058872, 'intertitres': 'Évolution en glissement annuel de l

In [52]:
print(answer["answer"])

La cause principale de l'augmentation de l'indice des prix à la consommation (IPC) est la hausse des prix des services. Selon les sources fournies, l'IPC a atteint des taux élevés en milieu d'année de 2008 (+ 3,6 % en juin et juillet), principalement en raison des fluctuations des prix des produits pétroliers, qui sont fortement liés aux prix du pétrole brut. Cependant, il faut noter que l'inflation reste principalement portée par l'accroissement des prix des services, et non pas par les prix des biens.

Cette information est tirée du contexte fourni, qui indique que les fluctuations de l'IPC sont principalement attribuées aux prix des services, et que les prix des produits pétroliers ont été une des causes des hausses de prix élevées en 2008.

Sources :
- "L’indice des prix à la consommation est l’instrument de mesure de l’inflation. Il permet d’estimer, entre deux périodes données, la variation du niveau général des prix des biens et des services proposés aux consommateurs sur le ter

### Adding a Reranker 

The goal of this part is to build a pipeline Langchain where we have added a reranker: a BM25, a ColBERT model, a french cross-encoder, a multilingual cross-encoder and several hyperparameters.  

Reranker model list : 
- multilingual cross encoder : BAAI/bge-reranker-large (multilingual),
- french cross encoder : antoinelouis/crossencoder-electra-base-french-mmarcoFR  OR dangvantuan/CrossEncoder-camembert-large
- BM25 : langchain_community.retrievers import BM25Retriever
- ColBERT : antoinelouis/colbertv2-camembert-L4-mmarcoFR


In [5]:
!mc cp s3/projet-llm-insee-open-data/data/chroma_database/chroma_db /home/onyxia/work/llm-open-data-insee/data --recursive

...ma.sqlite3: 2.08 GiB / 2.08 GiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 115.23 MiB/s 18s

In [3]:
sys.path.append("/home/onyxia/work/llm-open-data-insee/src") 

In [ ]:
from chain_building import (
    load_retriever
    )
from config import EMB_MODEL_NAME, MODEL_NAME

retriever = load_retriever(
                emb_model_name = EMB_MODEL_NAME,
                persist_directory="/home/onyxia/work/llm-open-data-insee/data/chroma_db",
                device="cuda",
                collection_name="insee_data"
            )


In [5]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [6]:
#test embedding retriever 
question = "Comment est calculé le pouvoir d'achat ?" 
#question = "Quelle est la cause principale de l'augmentation de l'indice des prix à la consommation (IPC)?"
results = retriever.invoke(question)
pretty_print_docs(results) #OK

Document 1:

Sujet de préoccupation des Français et élément du débat politique, le pouvoir d’achat suscite des controverses récurrentes, en partie liées à l’absence de définition partagée de ce terme. Il est certes facile de s’entendre sur le fait que le pouvoir d’achat doit rendre compte de l’évolution simultanée des revenus et des prix: si les revenus d’un ménage progressent plus rapidement que les prix, il gagne du pouvoir d’achat et pourra consommer ou épargner davantage; dans le cas contraire, il perd du pouvoir d’achat et doit diminuer sa consommation ou son épargne. La difficulté est de donner un sens précis à ces différents termes. La comptabilité nationale utilise des définitions qui ont l’avantage d’être cohérentes entre elles et harmonisées sur le plan international. Mais elle conduit à des évaluations qui sont souvent perçues comme trop optimistes, même lorsqu’on prend soin de bien différencier le pouvoir d’achat global de l’ensemble des ménages et le pouvoir d’achat par un

In [42]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder #CrossEncoder 
from ragatouille import RAGPretrainedModel #ColBERT
from langchain_community.retrievers import BM25Retriever #BM25

colBERT = RAGPretrainedModel.from_pretrained("antoinelouis/colbertv2-camembert-L4-mmarcoFR")
colBERT_retriever  = ContextualCompressionRetriever(base_compressor=colBERT.as_langchain_document_compressor(k=5), base_retriever=retriever)

compressed_docs = colBERT_retriever.invoke(question)
pretty_print_docs(compressed_docs)

/opt/mamba/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00, 21.05it/s]

Document 1:

.  À Mayotte, il est calculé à partir de la moyenne de deux estimations : par la demande (consommation finale, investissement et commerce extérieur) et par la production (valeurs ajoutées). Seule l’approche production est utilisée pour les comparaisons avec les régions de métropole, dont le PIB est calculé en régionalisant la valeur ajoutée. Le revenu disponible brut des ménages comprend les revenus d'activité, les revenus du patrimoine, les transferts en provenance d'autres ménages et les prestations sociales (y compris les pensions de retraite et les indemnités de chômage), nets des impôts directs. L'évolution du pouvoir d'achat du revenu disponible brut rapporte l'évolution du revenu disponible brut à celle du prix de la dépense de consommation finale des ménages, mesurée par l'évolution de l'indice des prix à la consommation. Pour approcher une notion plus individuelle du pouvoir d'achat, sa progression est rapportée à des unités démographiques (personne, ménage, unité

In [44]:
model = HuggingFaceCrossEncoder(model_name="dangvantuan/CrossEncoder-camembert-large") #"antoinelouis/crossencoder-electra-base-french-mmarcoFR")
compressor_1 = CrossEncoderReranker(model=model, top_n=5)
compression_retriever_1 = ContextualCompressionRetriever(
    base_compressor=compressor_1, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(question)
pretty_print_docs(compressed_docs)

/opt/mamba/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-06-24 10:23:53,091 - INFO - Use pytorch device: cuda


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Document 1:

L'Insee mesure l'évolution du pouvoir d'achat qui résulte de la différence entre l’évolution du revenu des ménages et l’évolution de l’indice des prix. Il mesure aussi la pauvreté et le niveau de vie des ménages. Il détermine le niveau de vie médian, le seuil de pauvreté ainsi que le nombre de personnes pauvres et analyse les caractéristiques de ces personnes.
----------------------------------------------------------------------------------------------------
Document 2:

. La taille de chaque ménage en nombre d'unités de consommation est calculée de la façon suivante: le premier adulte compte pour 1 unité de consommation(UC), chaque personne supplémentaire de 14ans et plus compte pour 0,5UC et chaque enfant de moins de 14ans compte pour 0,3UC. Le pouvoir d'achat par unité de consommation permet ainsi de prendre en considération non seulement le nombre de ménages, mais aussi l'évolution de la structure des ménages. C'est pourquoi il constitue l'indicateur qui est le plus p

In [45]:
model = HuggingFaceCrossEncoder(model_name= "BAAI/bge-reranker-large")
compressor_2 = CrossEncoderReranker(model=model, top_n=5)
compression_retriever_2 = ContextualCompressionRetriever(
    base_compressor=compressor_2, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(question)
pretty_print_docs(compressed_docs)

/opt/mamba/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-06-24 10:23:59,121 - INFO - Use pytorch device: cuda


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Document 1:

L'Insee mesure l'évolution du pouvoir d'achat qui résulte de la différence entre l’évolution du revenu des ménages et l’évolution de l’indice des prix. Il mesure aussi la pauvreté et le niveau de vie des ménages. Il détermine le niveau de vie médian, le seuil de pauvreté ainsi que le nombre de personnes pauvres et analyse les caractéristiques de ces personnes.
----------------------------------------------------------------------------------------------------
Document 2:

. La taille de chaque ménage en nombre d'unités de consommation est calculée de la façon suivante: le premier adulte compte pour 1 unité de consommation(UC), chaque personne supplémentaire de 14ans et plus compte pour 0,5UC et chaque enfant de moins de 14ans compte pour 0,3UC. Le pouvoir d'achat par unité de consommation permet ainsi de prendre en considération non seulement le nombre de ménages, mais aussi l'évolution de la structure des ménages. C'est pourquoi il constitue l'indicateur qui est le plus p

In [46]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(retrievers = [compression_retriever_1, compression_retriever_2, colBERT_retriever], weigths = [1/3,1/3,1/3])

compressed_docs = ensemble_retriever.invoke(question)
pretty_print_docs(compressed_docs)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00, 35.25it/s]

Document 1:

. La taille de chaque ménage en nombre d'unités de consommation est calculée de la façon suivante: le premier adulte compte pour 1 unité de consommation(UC), chaque personne supplémentaire de 14ans et plus compte pour 0,5UC et chaque enfant de moins de 14ans compte pour 0,3UC. Le pouvoir d'achat par unité de consommation permet ainsi de prendre en considération non seulement le nombre de ménages, mais aussi l'évolution de la structure des ménages. C'est pourquoi il constitue l'indicateur qui est le plus pertinent pour mesurer l'évolution du niveau de vie en moyenne sur l'ensemble des ménages. La dépense de consommation finale «pré-engagée» correspond aux dépenses qui sont réalisées dans le cadre d'un contrat difficilement renégociable à court terme. Elle comprend: Le revenu disponible «arbitrable» est défini comme le revenu disponible une fois déduit la dépense de consommation finale «pré-engagée». L'évolution du pouvoir d'achat «arbitrable» rapporte l'évolution du revenu 

In [36]:
from typing import Any, List, Optional, Sequence, Dict
from langchain.retrievers.document_compressors.base import BaseDocumentCompressor
from langchain.schema import Document
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableLambda, RunnableParallel , RunnablePassthrough
from langchain_community.retrievers import BM25Retriever

from langchain_core.callbacks import Callbacks
import math
from collections import Counter
from langchain_core.output_parsers import StrOutputParser

# Define the compression function
def compress_documents_lambda(documents: Sequence[Document], query: str, k: int = 5, **kwargs: Dict[str, Any]) -> Sequence[Document]:
    """Compress retrieved documents given the query context."""

    # Initialize the retriever with the documents
    retriever = BM25Retriever.from_documents(documents, k=k, **kwargs)
    relevant_docs = retriever.get_relevant_documents(query)
    return relevant_docs

# Define the complete chain
bm25_retriever = (
    RunnableParallel(
    {"documents": retriever, "query": RunnablePassthrough()}
    ) 
    | RunnableLambda(lambda r : compress_documents_lambda(documents= r["documents"] , query = r["query"]))
)

bm25_retriever.invoke(question)

[Document(page_content=".  À Mayotte, il est calculé à partir de la moyenne de deux estimations : par la demande (consommation finale, investissement et commerce extérieur) et par la production (valeurs ajoutées). Seule l’approche production est utilisée pour les comparaisons avec les régions de métropole, dont le PIB est calculé en régionalisant la valeur ajoutée. Le revenu disponible brut des ménages comprend les revenus d'activité, les revenus du patrimoine, les transferts en provenance d'autres ménages et les prestations sociales (y compris les pensions de retraite et les indemnités de chômage), nets des impôts directs. L'évolution du pouvoir d'achat du revenu disponible brut rapporte l'évolution du revenu disponible brut à celle du prix de la dépense de consommation finale des ménages, mesurée par l'évolution de l'indice des prix à la consommation. Pour approcher une notion plus individuelle du pouvoir d'achat, sa progression est rapportée à des unités démographiques (personne, mé

In [38]:
from langchain.retrievers import EnsembleRetriever
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder #CrossEncoder 

model = HuggingFaceCrossEncoder(model_name="dangvantuan/CrossEncoder-camembert-large") #"antoinelouis/crossencoder-electra-base-french-mmarcoFR")
compressor_1 = CrossEncoderReranker(model=model, top_n=5)

compression_retriever_cross_encoder = ContextualCompressionRetriever(
    base_compressor=compressor_1, base_retriever=retriever
)

emsemble_reranking = EnsembleRetriever(retrievers = [compression_retriever_cross_encoder, bm25_retriever], weigths = [0.5, 0.5])


/opt/mamba/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/809k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

2024-06-24 14:36:42,178 - INFO - Use pytorch device: cuda


In [39]:
emsemble_reranking.invoke(question)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[Document(page_content=". La taille de chaque ménage en nombre d'unités de consommation est calculée de la façon suivante: le premier adulte compte pour 1 unité de consommation(UC), chaque personne supplémentaire de 14ans et plus compte pour 0,5UC et chaque enfant de moins de 14ans compte pour 0,3UC. Le pouvoir d'achat par unité de consommation permet ainsi de prendre en considération non seulement le nombre de ménages, mais aussi l'évolution de la structure des ménages. C'est pourquoi il constitue l'indicateur qui est le plus pertinent pour mesurer l'évolution du niveau de vie en moyenne sur l'ensemble des ménages. La dépense de consommation finale «pré-engagée» correspond aux dépenses qui sont réalisées dans le cadre d'un contrat difficilement renégociable à court terme. Elle comprend: Le revenu disponible «arbitrable» est défini comme le revenu disponible une fois déduit la dépense de consommation finale «pré-engagée». L'évolution du pouvoir d'achat «arbitrable» rapporte l'évolution

### Adding a LLM Reranker

In [1]:
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    BitsAndBytesConfig,
    TextStreamer, 
    pipeline,
    TextStreamer
)
import torch
import torch.nn.functional as F

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

# quantization config 
quantization_config  = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype="float16",
            bnb_4bit_use_double_quant=False,
        )

config = AutoConfig.from_pretrained(model_id, trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config,
    config=config
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

#### Fine-grained label reranker 

In [72]:
# reranking utils function 

def expected_relevance_values(logits, grades_token_ids, list_grades):
    next_token_logits = logits[:, -1, :]
    next_token_logits = next_token_logits.cpu()[0]
    probabilities = F.softmax(next_token_logits[grades_token_ids], dim=-1).numpy()
    return np.dot(np.array(list_grades), probabilities)

def peak_relevance_likelihood(logits, grades_token_ids, list_grades):
    index_max_grade = np.array(list_grades).argmax()
    next_token_logits = logits[:, -1, :]
    probabilities = F.softmax(next_token_logits, dim=-1).cpu().numpy()[0]
    return probabilities[grades_token_ids[index_max_grade]]

def find_sublist_indices(main_list, sublist):
    sublist_length = len(sublist)
    main_list_length = len(main_list)
    
    # Helper function to check if sublist matches
    def is_sublist_at_index(index):
        return main_list[index:index + sublist_length] == sublist
    
    # Finding the first index
    first_index = -1
    
    for i in range(main_list_length - sublist_length + 1):
        if is_sublist_at_index(i):
            first_index = i
            break

    return first_index

In [ ]:
## assessing methods 
def RG_S(tokenizer, model,query, document, aggregating_method, k=5):

    list_grades = list(range(k))
    grades_token_ids = [tokenizer(str(grade))["input_ids"][1] for grade in list_grades]
 
    RG_S_template = """
    Sur une échelle de 0 à {k}, jugez la pertinence entre la requête et le document.
    Requête : {query}
    Document : {document}
    Réponse : """ 

    messages = [
        {"role": "system", "content": "Tu es un assistant chatbot expert en Statistique Publique."},
        {"role": "user", "content": RG_S_template.format(query=query, document=document, k=k)},
    ]

    input_text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False
    )

    inputs = tokenizer(input_text, return_tensors='pt').to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    return aggregating_method(logits, grades_token_ids, list_grades)


def RG_4L(tokenizer, model,query, document, args):
    possible_judgements = [" Parfaitement Pertinent", " Très Pertinent", " Assez Pertinent", " Non Pertinent"]
    list_grades = np.array([3, 2, 1, 0])
    RG_4L_template = """
    Evaluez la pertinence du document donné par rapport à la question posée.
    Répondez uniquement parmi : Parfaitement Pertinent, Très Pertinent, Assez Pertinent ou Non Pertinent.
    Requête : {query}
    Document : {document}
    Réponse : {judgement}"""

    messages = [
        {"role": "system", "content": "Tu es un assistant chatbot expert en Statistique Publique."},
        {"role": "user", "content": RG_4L_template},
    ]

    log_probs = []
    for judgement in possible_judgements:
        input_text = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=False,
            tokenize=False
        ).format(query=query, document=document, judgement=judgement)
        log_probs.append(compute_sequence_log_probs(sequence=input_text))

    probs = F.softmax(torch.tensor(log_probs), dim=-1).numpy()
    return np.dot(probs, list_grades)

def RG_3L(tokenizer, model,query, document, args):
    possible_judgements = [" Très Pertinent", " Assez Pertinent", " Non Pertinent"]
    list_grades = np.array([2, 1, 0])
    RG_3L_template = """
    Evaluez la pertinence du document donné par rapport à la question posée.
    Répondez uniquement parmi : Très Pertinent, Assez Pertinent ou Non Pertinent.
    Requête : {query}
    Document : {document}
    Réponse : {judgement}"""

    messages = [
        {"role": "system", "content": "Tu es un assistant chatbot expert en Statistique Publique."},
        {"role": "user", "content": RG_3L_template},
    ]

    log_probs = []
    for judgement in possible_judgements:
        input_text = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=False,
            tokenize=False
        ).format(query=query, document=document, judgement=judgement)
        log_probs.append(compute_sequence_log_probs(sequence=input_text))

    probs = F.softmax(torch.tensor(log_probs), dim=-1).numpy()
    return np.dot(probs, list_grades)
    
def RG_YN(tokenizer, model,query, document, aggregating_method):
    list_judgements = [" Oui", " Non"]
    grades_token_ids = [tokenizer(j)["input_ids"][1] for j in list_judgements]
    list_grades = [1, 0]

    RG_YN_template = """
    Pour la requête et le document suivants, jugez s'ils sont pertinents. Répondez UNIQUEMENT par Oui ou Non.
    Requête : {query}
    Document : {document}
    Réponse : """

    messages = [
        {"role": "system", "content": "Tu es un assistant chatbot expert en Statistique Publique."},
        {"role": "user", "content": RG_YN_template.format(query=query, document=document)},
    ]

    input_text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False
    )

    inputs = tokenizer(input_text, return_tensors='pt').to(model.device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    return aggregating_method(logits, grades_token_ids, list_grades)


In [73]:
def llm_reranking(tokenizer, model,  query, retrieved_documents, assessing_method, aggregating_method):
    docs_content = retrieved_documents.copy() #[doc.page_content for doc in retrieved_documents]

    scores = []
    for document in docs_content:  
        score = assessing_method(tokenizer, model, query, document, aggregating_method)
        scores.append(score)

    docs_with_scores = list(zip(retrieved_documents, scores))
    docs_with_scores.sort(key=lambda x: x[1], reverse=True)
    sorted_documents = [doc for doc, score in docs_with_scores] #docs_with_scores 
    return sorted_documents

In [4]:
expected_relevance_values(query="Combien y a t il d'habitant en France en 2024?", document="Le nombre d'habitant en Birmanie est de 54 millions de personnes")

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


0.1433243912179023

In [75]:
question = "Comment le taux de chômage en France a-t-il évolué au cours des dix dernières années ?"

documents = [
    "Le taux de chômage en France a connu des variations significatives au cours des dix dernières années. En 2014, le taux de chômage s'élevait à environ 10 %. Après un pic en 2015 à près de 10,5 %, il a progressivement diminué pour atteindre 8 % en 2019. La crise sanitaire de 2020 a provoqué une hausse temporaire du chômage, mais les réformes économiques ont permis de ramener le taux à 7,8 % en 2023.",
    "Selon les données de l'INSEE, le taux de chômage en France a fluctué entre 10 % et 7,8 % au cours des dix dernières années. Après une hausse notable en 2015, des mesures gouvernementales ont contribué à une baisse progressive. Cependant, la pandémie de COVID-19 en 2020 a inversé cette tendance temporairement avant de redescendre en 2023.",
    "Entre 2014 et 2023, le taux de chômage en France a varié considérablement. Après avoir atteint un sommet de 10,5 % en 2015, il a progressivement diminué, atteignant un minimum de 7,8 % en 2023. La pandémie de COVID-19 a temporairement perturbé cette tendance, augmentant le taux de chômage en 2020 et 2021.",
    "Au cours des dix dernières années, le taux de chômage en France a montré une tendance à la baisse. Après avoir atteint un pic en 2015, il a progressivement diminué, bien que la crise sanitaire de 2020 ait causé une augmentation temporaire. En 2023, le taux de chômage était de 7,8 %.",
    "L'évolution du taux de chômage en France de 2014 à 2023 révèle une baisse progressive après un pic en 2015. Bien que la pandémie ait provoqué une hausse temporaire, le taux de chômage a repris sa tendance à la baisse pour atteindre 7,8 % en 2023, selon l'INSEE.",
    "La cuisine française est mondialement reconnue pour sa diversité et son raffinement. Des plats emblématiques comme le coq au vin, la bouillabaisse et le bœuf bourguignon illustrent la richesse gastronomique du pays. Les vins et fromages français sont également très appréciés à l'international.",
    "Le système éducatif français est structuré en plusieurs niveaux, allant de l'école maternelle à l'université. L'éducation est obligatoire de 3 à 16 ans. Les élèves passent des examens nationaux comme le brevet des collèges et le baccalauréat, qui sont des étapes clés dans leur parcours scolaire.",
    "La France est l'un des leaders mondiaux dans la production d'énergie nucléaire. Environ 70 % de l'électricité du pays provient de centrales nucléaires. Cette dépendance permet à la France d'avoir une empreinte carbone relativement faible par rapport à d'autres pays européens.",
    "Le football est le sport le plus populaire en France, avec des millions de licenciés et de nombreux clubs répartis sur tout le territoire. L'équipe nationale, les Bleus, a remporté la Coupe du Monde de la FIFA en 1998 et 2018, ce qui a renforcé l'engouement pour ce sport parmi les Français.",
    "La France est riche en patrimoine culturel, avec des monuments emblématiques tels que la Tour Eiffel, le Mont Saint-Michel et le Château de Versailles. Le pays abrite également de nombreux musées de renommée mondiale, dont le Louvre et le Musée d'Orsay, qui attirent des millions de visiteurs chaque année."
]

dict_doc_rank = {doc : r for r, doc in enumerate(documents)}

for ass_func in [RG_YN, RG_S, RG_3L, RG_4L]:
    print("------------------------")
    reranked_documents = llm_reranking(
        query=question, 
        retrieved_documents=documents, 
        assessing_method=ass_func, 
        aggregating_method=expected_relevance_values
        )
    for i in range(5):
        print(f"Doc {i} th", reranked_documents[i])
        

------------------------
Doc 0 th L'évolution du taux de chômage en France de 2014 à 2023 révèle une baisse progressive après un pic en 2015. Bien que la pandémie ait provoqué une hausse temporaire, le taux de chômage a repris sa tendance à la baisse pour atteindre 7,8 % en 2023, selon l'INSEE.
Doc 1 th Au cours des dix dernières années, le taux de chômage en France a montré une tendance à la baisse. Après avoir atteint un pic en 2015, il a progressivement diminué, bien que la crise sanitaire de 2020 ait causé une augmentation temporaire. En 2023, le taux de chômage était de 7,8 %.
Doc 2 th Selon les données de l'INSEE, le taux de chômage en France a fluctué entre 10 % et 7,8 % au cours des dix dernières années. Après une hausse notable en 2015, des mesures gouvernementales ont contribué à une baisse progressive. Cependant, la pandémie de COVID-19 en 2020 a inversé cette tendance temporairement avant de redescendre en 2023.
Doc 3 th Entre 2014 et 2023, le taux de chômage en France a va

#### Experiment LLM reranker 